# NPS verbatim sentiment analysis using Amazon Bedrock

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Overview

In this notebook, you are going to ingest a csv file for jNPS data from an Amazon S3 bucket, run the topic extraction and sentiment analysis on the data in the NPS verbatim column, and finally export the data back into an Amazon S3 bucket for further analysis.

### Architecture

![](../Images/Heartbeat-POC-Architecture.png)

In this architecture:

1. A raw NPS csv file is loaded
1. A foundation model processes the NPS verbatim data
1. Model returns a response with the sentiment analysis against the topics mentioned in the NPS verbatim column.

### Use case

The use case here is NPS verbatim analysis in order to improve customer experience and operational effectiveness.
This approach can also be used to analyze call transcripts, chat transcripts, journey events to create journey summaries etc.

### Challenges

This approach can be used when the input text or file fits within the model context length. If the input text including the prompts exceed the context window size of the model, a retrieval augmented approach could be applied for topic identification, followed by sentiment analysis.

## Setup

### Install relevant modules

In [ ]:
%pip install --no-build-isolation --force-reinstall --quiet\
    "boto3>=1.28.57" \
    "botocore>=1.31.57"\
    "awswrangler"

### Import relevant modules

In [9]:
import json
import os
import sys
import boto3
import awswrangler as wr
import pandas as pd

### Initiate Amazon Bedrock Client

In [4]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: MIT-0
"""Helper utilities for working with Amazon Bedrock from Python notebooks"""
# Python Built-Ins:
import os
from typing import Optional

# External Dependencies:
import boto3
from botocore.config import Config


def get_bedrock_client(
    assumed_role: Optional[str] = None,
    region: Optional[str] = None,
    runtime: Optional[bool] = True,
):
    """Create a boto3 client for Amazon Bedrock, with optional configuration overrides

    Parameters
    ----------
    assumed_role :
        Optional ARN of an AWS IAM role to assume for calling the Bedrock service. If not
        specified, the current active credentials will be used.
    region :
        Optional name of the AWS Region in which the service should be called (e.g. "us-east-1").
        If not specified, AWS_REGION or AWS_DEFAULT_REGION environment variable will be used.
    runtime :
        Optional choice of getting different client to perform operations with the Amazon Bedrock service.
    """
    if region is None:
        target_region = os.environ.get("AWS_REGION", os.environ.get("AWS_DEFAULT_REGION"))
    else:
        target_region = region

    print(f"Create new client\n  Using region: {target_region}")
    session_kwargs = {"region_name": target_region}
    client_kwargs = {**session_kwargs}

    profile_name = os.environ.get("AWS_PROFILE")
    if profile_name:
        print(f"  Using profile: {profile_name}")
        session_kwargs["profile_name"] = profile_name

    retry_config = Config(
        region_name=target_region,
        retries={
            "max_attempts": 10,
            "mode": "standard",
        },
    )
    session = boto3.Session(**session_kwargs)

    if assumed_role:
        print(f"  Using role: {assumed_role}", end='')
        sts = session.client("sts")
        response = sts.assume_role(
            RoleArn=str(assumed_role),
            RoleSessionName="langchain-llm-1"
        )
        print(" ... successful!")
        client_kwargs["aws_access_key_id"] = response["Credentials"]["AccessKeyId"]
        client_kwargs["aws_secret_access_key"] = response["Credentials"]["SecretAccessKey"]
        client_kwargs["aws_session_token"] = response["Credentials"]["SessionToken"]

    if runtime:
        service_name='bedrock-runtime'
    else:
        service_name='bedrock'

    bedrock_client = session.client(
        service_name=service_name,
        config=retry_config,
        **client_kwargs
    )

    print("boto3 Bedrock client successfully created!")
    print(bedrock_client._endpoint)
    return bedrock_client


### Initiate Amazon Bedrock Client

In [6]:
# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


## Calling the Amazon Bedrock API
 
To learn detail of API request to Amazon Bedrock, this notebook introduces how to create API request and send the request via Boto3 rather than relying on langchain, which gives simpler API by wrapping Boto3 operation. 

### Request Syntax of InvokeModel in Boto3


We use `InvokeModel` API for sending request to a foundation model. Here is an example of API request for sending text to Anthropic Claude. Inference parameters in `textGenerationConfig` depends on the model that you are about to use. Inference paramerters of Anthropic Claude are:

- **temperature** tunes the degree of randomness in generation. Lower temperatures mean less random generations.
- **top_p** less than one keeps only the smallest set of most probable tokens with probabilities that add up to top_p or higher for generation.
- **top_k** can be used to reduce repetitiveness of generated tokens. The higher the value, the stronger a penalty is applied to previously present tokens, proportional to how many times they have already appeared in the prompt or prior generation.
- **max_tokens_to_sample** is maximum number of tokens to generate. Responses are not guaranteed to fill up to the maximum desired length.
- **stop_sequences** are sequences where the API will stop generating further tokens. The returned text will not contain the stop sequence.

```python
response = bedrock.invoke_model(body=
                                {"prompt":"this is where you place your input text",
                                 "max_tokens_to_sample":4096,
                                 "temperature":0.5,
                                 "top_k":250,
                                 "top_p":0.5,
                                 "stop_sequences":[]
                                },
                                modelId="anthropic.claude-v2", 
                                accept=accept, 
                                contentType=contentType)

```

### Load and explore the verbatim file

⚠️ ⚠️ ⚠️ Please replace the below `bucket` and `key` variables below with the S3 bucket and prefix from your account where the jNPS file is stored ⚠️ ⚠️ ⚠️

In [10]:
# S3 location
bucket = 'vodacom-nps-poc-ajay'
key = 'raw-data/September2023/Buy_SetUp.csv'

# Read CSV using awswrangler 
df = wr.s3.read_csv(path=[f's3://{bucket}/{key}'])

#### Explore the NPS verbatims

In [ ]:
for index, row in df.iterrows():
    if not pd.isnull(row['VERBATIM']):
        print(f"Row {index} : {row['VERBATIM']}")
        if index>50:
            break

#### Define a function to construct the prompt for the LLM

In [12]:
def generate_prompt(text):
    prompt = """Human: ‘The text below is a verbatim response to an NPS survey question -  
                
                "Is there anything else you would like to share with us about your experience with Vodacom?"
                
                Pick the topics mentioned in the text from a reference list of topics provided below. For each topic, respond with the sentiment expressed against that topic.

                Text:"""+text+"""

                List of Topics:

                Change Account Details, 
                MSISDN change,
                proof of usage,
                Transfer of Ownership, 
                Account  Payment, 
                Account - Payment,
                Billing Enquiry,
                Call Barring,
                Itemised Billing,
                Payment Arrangement,
                Car phone,
                competitors,
                Emails,
                Vodacom,
                Cancellation Enquiry,
                MNP Port Enquiry,
                PIN / PUK Enquiry,
                Customer Care enquiry,
                Customer's Feeling,
                abusive call,
                abusive text,
                abusive txt,
                Blacklist/Unblacklist Device,
                Delivery Enquiry,
                Handset Enquiry,
                harassing,
                harassment,
                insurance,
                malicious,
                nuisance,
                proof of purchase,
                return,
                stolen,
                unwanted,
                Upgrade Enquiry,
                Vodacom Repair - Complaint,
                General Network,
                admin fee,
                Authentication,
                equest for Credit Refund,
                Fraud,
                Proof of Payment,
                Charge Dispute,
                Data Bundle Enquiry,
                Deal Match,
                discount,
                Tariff Enquiry,
                Tariff Enquiry/Free change,
                Contract,
                entertainment packs,
                International Roaming Enquiry,
                my vodafone family,
                prepaid,
                top up,
                Account Query,
                complaint,
                Response,
                Service,
                buy back,
                content control,
                ivr,
                live chat,
                Locking/Unlocking,
                My Vodacom app,
                System: Down/Not Available,
                USSD Enquiry,
                Vodacom Online,
                voicemail,
                activation,
                sim,
                assistance & helpfulness,
                Broken Arrangement,
                call back,
                Call Transferred,
                communication,
                Dropped Calls,
                efficiency,
                explanation,
                friendliness & care,
                hold,
                inconsistency,
                language,
                listening,
                misinformation,
                politeness,
                previous staff behavior,
                professionalism,
                Sales Enquiry,
                staff behavior,
                understanding,
                Vodacom Shop,
                wait time

                If the text is not relevant to a topic from the list above, but another relevant topic is found, come up with a new topic and highlight it as a new topic.
                
                If no topic is found then respond with Topic as "none" and sentiment as the sentiment of the text.
                
                Respond with sentiment with options Positive/Negative/Neutral.

                Here are a few examples of text and expected responses. Follow the format of responses mentioned in the examples:

                Example 1:

                Text: I like to go to my local Vodacom shop(Stillbay). Friendly people and helpfull. The call centre person on the other hand are a bit pushy, maybe the client can get SMS to state that there upgrade is due and then reply if they will visit a branch or if they want a agent giving them a call. Give us the choice.

                Response
                [
                  {
                    "Topic": "Vodacom Shop",
                    "Sentiment": "Positive"
                  },
                  {
                    "Topic": "Customer Care enquiry", 
                    "Sentiment": "Negative"
                  },
                  {
                    "Topic": "Upgrade Process",
                    "Sentiment": "Neutral",
                    "Note": "New"
                  }
                ]

                Example 2:
                Text: Vodacom has become unaffordable on calls and data. Network also is a problem in many places
                Response:
                [
                  {
                    "Topic": "Affordability", 
                    "Sentiment": "Negative",
                    "Note": "New"
                  },
                  {
                    "Topic": "General Network",
                    "Sentiment": "Negative"
                  }
                ]
                
                Example 3:
                Text: No
                Response:
                [
                  {
                    "Topic": "none", 
                    "Sentiment": "Neutral"
                  }
                ]
                
                Example 4:
                Text: Yes
                Response:
                [
                  {
                    "Topic": "none", 
                    "Sentiment": "Neutral"
                  }
                ]
                
                Example 5:
                Text: All is well
                Response:
                [
                  {
                    "Topic": "Customer's Feeling", 
                    "Sentiment": "Positive"
                  }
                ]
                
                Example 6:
                Text: Everything is bad
                Response:
                [
                  {
                    "Topic": "Customer's Feeling", 
                    "Sentiment": "Negative"
                  }
                ]

                Assistant:"""
    
    body = json.dumps({"prompt": prompt,
                 "max_tokens_to_sample":4096,
                 "temperature":0.5,
                 "top_k":250,
                 "top_p":0.5,
                 "stop_sequences":[]
                  }) 
    return body


## Sentiment Analysis with Amazon Bedrock

### Helper functions

#### This function populates the result data frame in the required format

In [13]:
import pandas as pd

def update_df(dict_list, index, df, df_insights):
            
    row = df.iloc[index]
    row_copy = row.copy()
    #dict_list = json.loads(dict_list)
    
    for d in dict_list:
        print(d)
        # Use concat instead of append  
        df_insights = pd.concat([df_insights, row_copy.to_frame().T], ignore_index=True)
        
         # Add new columns if needed
        cols = ['Topic', 'Sentiment', 'Is_New_Topic']
        for col in cols:
            if col not in df_insights.columns:
                df_insights[col] = ""
            
        df_insights.at[df_insights.index[-1], 'Topic'] = d.get('Topic')
        df_insights.at[df_insights.index[-1], 'Sentiment'] = d.get('Sentiment')
        df_insights.at[df_insights.index[-1], 'Is_New_Topic'] = d.get('Note') == 'New'

    return df_insights

#### This function finds and extracts the json from the LLM response 

In [14]:
import re
import json

def find_and_extract_json(text):
    # Use regex to find text matching JSON structure
    match = re.search(r'\[.*?\]', text, re.DOTALL)
    
    if match:
        # Extract the matched JSON string 
        json_str = match.group(0)  
        
        try:
            # Try loading it as JSON 
            data = json.loads(json_str)
            return data
        except json.JSONDecodeError:
            # Was not valid JSON
            return None
        
    return None

#### This function calls the Bedrock API to derive targetted sentiment against mentioned topics from the NPS verbatim

In [15]:
def extract_nps_verbatim_insights_fs(body,modelId, df, df_insights, index):
    modelId = modelId # change this to use a different version from the model provider
    accept = 'application/json'
    contentType = 'application/json'

    response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())
    
    completion = response_body['completion']
    
    completion_json = find_and_extract_json(completion)

    df_insights_upd = update_df(completion_json, index, df, df_insights)
    
    return df_insights_upd

#### These lines of code iterate through the ingested NPS verbatim file, calls the nps insights extraction, and records the output

> *This cell will take about 15min to run

In [ ]:
df_insights = pd.DataFrame()
for index, row in df.iterrows():
    if not pd.isnull(row['VERBATIM']):
        raw_verbatim = row['VERBATIM']
        print(raw_verbatim+"\n ********************************\n")
        prompt = generate_prompt(raw_verbatim)
        #print(prompt)
        df_insights_upd = extract_nps_verbatim_insights_fs(prompt,'anthropic.claude-v2',df,df_insights,index)
        df_insights = df_insights_upd
        #print(f"Row {index} : {nps_insight} \n ********************************\n")

#### Explore the NPS insights output

In [ ]:
# Select columns to view
columns = ['VERBATIM', 'Topic','Sentiment', 'Is_New_Topic']

# View first 5 rows for selected columns
print(df_insights_upd.loc[:100, columns])

#### Upload output nps insights file to S3
⚠️ ⚠️ ⚠️ Please replace the below `bucket` and `key` variables below with the S3 bucket and prefix from your account where the jNPS sentiment analysis file should be stored ⚠️ ⚠️ ⚠️

In [ ]:
# S3 location
bucket = 'vodacom-nps-poc-ajay'
key = 'nps-insights/September2023/Buy_SetUp_insights.csv'

# Write DataFrame to S3 in CSV format
wr.s3.to_csv(
    df=df_insights_upd,
    path=f"s3://{bucket}/{key}", 
    index=False  
)

## Conclusion
You have now successfully used Amazon Bedrock to analyze NPS verbatims in the jNPS input file provided. Please go ahead and analyze the output using Amazon Quicksight's Generative BI.

## Thank You